### Dependencies

In [16]:
import pandas as pd
import matplotlib.pyplot as plt

from time import strftime

import warnings
warnings.filterwarnings("ignore")

### Needed Functions

In [2]:
def sheetsTTC(xlsx_file):
    
    """ 
    Reads in an Excel file and concatenates each sheet into 
    a returned pandas DataFrame.    
    
    Note:
        - Requires: pandas
        - Requires: Each worksheet must have exact matching
                    n columns, column order, column labels
                    and column dtypes.
    """
    
    xls = pd.ExcelFile(xlsx_file)
    
    out_df = pd.DataFrame()
    
    for sheet in xls.sheet_names:
        
        df = pd.read_excel(xls, sheet_name = sheet)
        
        df = df[[
            'Report Date', 'Route', 'Time', 'Day', 'Location', 'Incident', 'Min Delay', 'Min Gap', 'Direction', 'Vehicle'
        ]]
        
        out_df = pd.concat([out_df, df])
        
    return out_df

def excelDate(excel_time):
    
    """
    Converts excel datetime float format to pandas datetime
    
    """
    
    return pd.to_datetime('1900-01-01') + pd.to_timedelta(excel_time, 'D')


### Buses

In [3]:
# Bus 2019
bus2019_xls = pd.ExcelFile('Bus_2019.xlsx')

jan = pd.read_excel(bus2019_xls, 'Jan 2019')
feb = pd.read_excel(bus2019_xls, 'Feb 2019')
mar = pd.read_excel(bus2019_xls, 'Mar 2019')
apr = pd.read_excel(bus2019_xls, 'Apr 2019')
may = pd.read_excel(bus2019_xls, 'May 2019')

# April is the only month in the 5 1/2 years of data
# with the column 'Incident ID', so it gets dropped
apr = apr.drop(columns={'Incident ID'})

# Rename the two mis-labelled columns
apr = apr.rename(columns={'Delay': 'Min Delay', 'Gap': 'Min Gap'})

bus19 = pd.concat([jan, feb])
bus19 = pd.concat([bus19, mar])
bus19 = pd.concat([bus19, apr])
bus19 = pd.concat([bus19, may])

# Apply sheetsTTC Function
bus18 = sheetsTTC('Bus_2018.xlsx')
bus17 = sheetsTTC('Bus_2017.xlsx')
bus16 = sheetsTTC('Bus_2016.xlsx')
bus15 = sheetsTTC('Bus_2015.xlsx')
bus14 = sheetsTTC('Bus_2014.xlsx')

# Add Year Columns
bus19['Year'] = 2019
bus18['Year'] = 2018
bus17['Year'] = 2017
bus16['Year'] = 2016
bus15['Year'] = 2015
bus14['Year'] = 2014

# Combine 2014 - 2019
buses = pd.concat([bus19, bus18])
buses = pd.concat([buses, bus17])
buses = pd.concat([buses, bus16])
buses = pd.concat([buses, bus15])
buses = pd.concat([buses, bus14])

# Add Bus labeled Type column
buses['Type'] = 'Bus'

In [4]:
buses.head()

Report Date  Route      Time      Day                     Location  \
0        43466     39  0.009028  Tuesday                         NECR   
1        43466    111  0.010417  Tuesday                    Eglington   
2        43466     35  0.012500  Tuesday                        Finch   
3        43466     25  0.020833  Tuesday  Don Mills Rd/Eglinton Ave E   
4        43466     36  0.027778  Tuesday                   Humberwood   

        Incident  Min Delay  Min Gap Direction  Vehicle  Year Type  
0     Mechanical        9.0     18.0       W/B   1794.0  2019  Bus  
1     Mechanical       15.0     30.0       S/B   8065.0  2019  Bus  
2     Mechanical        9.0     18.0       S/B   3275.0  2019  Bus  
3     Mechanical        9.0     18.0       N/B   8840.0  2019  Bus  
4  Investigation        9.0     18.0       E/B   9119.0  2019  Bus

### Streetcar

In [5]:
# Apply sheetsTTC Function
streetcar19 = sheetsTTC('Streetcar_2019.xlsx')
streetcar18 = sheetsTTC('Streetcar_2018.xlsx')
streetcar17 = sheetsTTC('Streetcar_2017.xlsx')
streetcar16 = sheetsTTC('Streetcar_2016.xlsx')
streetcar15 = sheetsTTC('Streetcar_2015.xlsx')
streetcar14 = sheetsTTC('Streetcar_2014.xlsx')

# Add Year Columns
streetcar19['Year'] = 2019
streetcar18['Year'] = 2018
streetcar17['Year'] = 2017
streetcar16['Year'] = 2016
streetcar15['Year'] = 2015
streetcar14['Year'] = 2014

# Combine 2014 - 2019
streetcars = pd.concat([streetcar19, streetcar18])
streetcars = pd.concat([streetcars, streetcar17])
streetcars = pd.concat([streetcars,streetcar16])
streetcars = pd.concat([streetcars, streetcar15])
streetcars = pd.concat([streetcars, streetcar14])

# Add Streetcar labeled Type column
streetcars['Type'] = 'Streetcar'

In [6]:
streetcars.head()

Report Date  Route      Time      Day              Location       Incident  \
0        43466    301  0.047222  Tuesday       Queen/Braodview        Held By   
1        43466    511  0.062500  Tuesday      Bathurst/College  Investigation   
2        43466    306  0.069444  Tuesday      Dundas West stn.     Mechanical   
3        43466    505  0.131944  Tuesday  Lansdowne and Dundas     Mechanical   
4        43466    310  0.163194  Tuesday  Spadina and Lakshore        Held By   

   Min Delay  Min Gap Direction  Vehicle  Year       Type  
0        6.0     13.0       E/B   4193.0  2019  Streetcar  
1        5.0     10.0       N/B   1038.0  2019  Streetcar  
2        8.0     16.0       W/B   4146.0  2019  Streetcar  
3        6.0     12.0       E/B   8416.0  2019  Streetcar  
4       20.0     30.0       N/B   4465.0  2019  Streetcar

### Surface Routes (Streetcar + Buses)

In [18]:
surface = pd.concat([buses, streetcars])

In [20]:
surface['Report Date'] = excelDate(surface['Report Date'])
surface['Time'] = round(((surface['Time'] * 86400)/60)/60, 2)
surface['Time'] = time(surface['Time']).strftime('%-H')

NameError: name 'time' is not defined

In [9]:
surface.to_csv('ttc_Delays_surfaceRoutes.csv')
surface.sample(25)

Report Date  Route   Time        Day                 Location  \
5256  2014-09-20    195  15.00   Thursday             Entire Route   
2282  2015-08-17     83  10.60   Saturday         Lakeshore Garage   
465   2016-05-05    199  16.48    Tuesday            Finch Station   
1702  2018-03-13     86  16.93     Sunday    Eglinton and Barbados   
1962  2015-02-08    165  21.83     Friday        weston and rueben   
3627  2018-05-18    710   9.00  Wednesday             Eglinton stn   
130   2014-07-04    107   6.30  Wednesday            Downsview stn   
157   2014-08-03    116  15.47     Friday          Kennedy Station   
5122  2018-01-25     70  16.27    Tuesday         O'Connor/Sunrise   
3278  2018-10-19     87  17.93  Wednesday                 Main STN   
7614  2014-11-01     36  15.80   Thursday                 Bathurst   
1619  2016-03-09    195   9.58     Monday  Murray Ross and Steeles   
3599  2014-04-16    100   8.72     Monday        Broadview Station   
5881  2014-06-25     96  18.10     Monday             Edgar/Wilson   
1522  2017-08-11    112  14.67  Wednesday              kipling stn   
4198  2017-08-24     33  14.67    Tuesday   St.Cllair west station   
5079  2016-02-29     87  11.42   Saturday                main stn.   
63    2016-10-05    500   6.02     Monday       Queen @! Connaught   
5852  2014-01-22    199  17.35     Monday                Finch Stn   
690   2018-11-06    606  19.08     Sunday           Warden station   
3232  2016-03-18    199  12.32  Wednesday            Finch station   
6335  2015-02-01     51   8.00     Friday         Eglinton Station   
7424  2014-05-01     68  14.82    Tuesday                   Garage   
1956  2015-05-11     80  19.50   Saturday        Kipling/Queensway   
3415  2016-02-20    195   7.35   Thursday                     JSTE   

                 Incident  Min Delay  Min Gap Direction  Vehicle  Year  \
5256        General Delay       15.0     20.0        BW      1.0  2014   
2282           Mechanical       24.0     48.0        NB   7801.0  2015   
465            Mechanical       12.0     24.0       W/B   1127.0  2016   
1702        Investigation        8.0     15.0       W/B   8852.0  2018   
1962           Mechanical        8.0     16.0       e/b   1286.0  2015   
3627   Utilized Off Route        1.0      1.0       N/B   8755.0  2018   
130   Late Leaving Garage        7.0     22.0       n/b   1828.0  2014   
157    Utilized Off Route        4.0      8.0        NB   7645.0  2014   
5122        Investigation       17.0     34.0       N/B   8780.0  2018   
3278           Mechanical        5.0     10.0       W/B   8944.0  2018   
7614        Investigation        4.0      8.0         E   1306.0  2014   
1619        Investigation       10.0     20.0         S   8480.0  2016   
3599   Utilized Off Route        3.0      6.0         N   7436.0  2014   
5881        Investigation       17.0     34.0         w   1258.0  2014   
1522        Investigation       24.0     48.0       N/B   7969.0  2017   
4198   Utilized Off Route       20.0     40.0         N   8389.0  2017   
5079   Utilized Off Route       10.0     20.0        wb   7746.0  2016   
63             Mechanical        5.0     10.0       E/B   4026.0  2016   
5852           Mechanical        5.0      9.0       W/B   7047.0  2014   
690    Utilized Off Route        1.0      1.0       S/B   8541.0  2018   
3232   Utilized Off Route        4.0      8.0        EB   1769.0  2016   
6335        General Delay       20.0     20.0        NB      NaN  2015   
7424  Late Leaving Garage        9.0     18.0       S/B   7509.0  2014   
1956           Mechanical       30.0     60.0         E   7322.0  2015   
3415  Late Leaving Garage        8.0     16.0         S   1379.0  2016   

           Type  
5256        Bus  
2282        Bus  
465         Bus  
1702        Bus  
1962        Bus  
3627        Bus  
130         Bus  
157         Bus  
5122        Bus  
3278        Bus  
7614        Bus  
1619        Bus  
3599        Bus  
5881   

### Quick Look

In [10]:
surface.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494169 entries, 0 to 873
Data columns (total 12 columns):
Report Date    494169 non-null datetime64[ns]
Route          494169 non-null int64
Time           494169 non-null float64
Day            494169 non-null object
Location       493238 non-null object
Incident       493238 non-null object
Min Delay      493855 non-null float64
Min Gap        493392 non-null float64
Direction      484880 non-null object
Vehicle        428708 non-null float64
Year           494169 non-null int64
Type           494169 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(5)
memory usage: 49.0+ MB


In [11]:
surface.describe()

Route          Time      Min Delay        Min Gap  \
count  494169.000000  4.941690e+05  493855.000000  493392.000000   
mean      167.907617  6.596399e+01      18.715536      27.504749   
std       248.292435  7.378415e+03     356.612916      67.871773   
min         1.000000  0.000000e+00     -54.000000      -2.000000   
25%        42.000000  7.300000e+00       6.000000      11.000000   
50%        87.000000  1.275000e+01      10.000000      18.000000   
75%       191.000000  1.617000e+01      15.000000      28.000000   
max    106117.000000  1.046256e+06  246245.000000    6528.000000   

             Vehicle           Year  
count  428708.000000  494169.000000  
mean     5410.035042    2016.142395  
std      3123.756859       1.597574  
min         0.000000    2014.000000  
25%      1669.000000    2015.000000  
50%      7428.000000    2016.000000  
75%      8145.000000    2018.000000  
max    163242.000000    2019.000000

In [12]:
surface['Incident'].value_counts()

Mechanical                          192907
Late Leaving Garage                  72983
Utilized Off Route                   70781
General Delay                        64670
Investigation                        48086
Diversion                            24870
Emergency Services                   10336
Held By                               8321
Overhead - Pantograph                  142
Vision                                  61
Late Leaving Garage - Mechanical        48
Late Leaving Garage - Operator          25
Late Leaving Garage - Management         8
Name: Incident, dtype: int64

In [14]:
surface['Type'].value_counts()

Bus          421625
Streetcar     72544
Name: Type, dtype: int64

In [15]:
surface['Day'].value_counts()

Thursday     82791
Tuesday      82461
Wednesday    81684
Friday       80781
Monday       74617
Saturday     49662
Sunday       42173
Name: Day, dtype: int64